In [ ]:
from pyspark import SparkContext

In [6]:
def fMapper(value):
    user,friends = value.split(" ")
    keyList = []
    
    for friend in friends:
        keyList.append((''.join(sorted(user + friend)), friends.replace(friend, "")))
        # Appends the user and the first friend as the key and deletes the friend from the friends list 
        # and adds the remaining friends to the value part of the list
    return keyList

In [7]:
def fReducer(key, friends):
    reduce = ''
    for x in key:
        if x in friends:
            reduce = reduce + x 
    return reduce

In [11]:
data = sc.textFile('facebook_combined.txt',1)
mappedData = data.flatMap(fMapper)
#print(mappedData.take(500))
reducedData = mappedData.reduceByKey(fReducer)

reducedData.coalesce(1).saveAsTextFile('commonFriends')
# print(reducedData.take(500))
print('Done')

Done


In [73]:
from pyspark.sql import SparkSession
from pyspark.sql import *
from pyspark.sql.types import IntegerType()

df = spark.read.csv("WorldCupMatches.csv",header=True).withColumnRenamed('Home Team Name', 'Home_Team')
    .withColumnRenamed('Away Team Name', 'Away_Team').withColumnRenamed('Away Team Goals', 'Away_Team_Goals')
    .withColumnRenamed('Home Team Goals', 'Home_Team_Goals').withColumnRenamed('Win conditions', 'Win_conditions')
    .withColumnRenamed('Half-time Home Goals', 'Half_time_Home_Goals')
    .withColumnRenamed('Half-time Away Goals', 'Half_time_Away_Goals').withColumnRenamed('Assistant 1', 'Assistant_1')
    .withColumnRenamed('Assistant 2', 'Assistant_2').withColumnRenamed('Home Team Initials', 'Home_Team_Initials')
    .withColumnRenamed('Away Team Initials', 'Away_Team_Initials')


df=df.withColumn('Away_Team_Goals',df['Away_Team_Goals'].cast(IntegerType()))
df=df.withColumn('Home_Team_Goals',df['Home_Team_Goals'].cast(IntegerType()))
print(df.schema)

StructType(List(StructField(Year,StringType,true),StructField(Datetime,StringType,true),StructField(Stage,StringType,true),StructField(Stadium,StringType,true),StructField(City,StringType,true),StructField(Home_Team,StringType,true),StructField(Home_Team_Goals,DoubleType,true),StructField(Away_Team_Goals,DoubleType,true),StructField(Away_Team,StringType,true),StructField(Win_conditions,StringType,true),StructField(Attendance,StringType,true),StructField(Half_time_Home_Goals,StringType,true),StructField(Half_time_Away_Goals,StringType,true),StructField(Referee,StringType,true),StructField(Assistant_1,StringType,true),StructField(Assistant_2,StringType,true),StructField(RoundID,StringType,true),StructField(MatchID,StringType,true),StructField(Home_Team_Initials,StringType,true),StructField(Away_Team_Initials,StringType,true)))


In [48]:
#1 
df.filter(df.Year.like("1958")).show()
# This returns all the matches that were played in 1958 

+----+--------------------+-------+---------------+------------+----------------+---------------+---------------+----------------+--------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+-------+-------+------------------+------------------+
|Year|            Datetime|  Stage|        Stadium|        City|       Home_Team|Home_Team_Goals|Away_Team_Goals|       Away_Team|Win_conditions|Attendance|Half_time_Home_Goals|Half_time_Away_Goals|             Referee|         Assistant_1|         Assistant_2|RoundID|MatchID|Home_Team_Initials|Away_Team_Initials|
+----+--------------------+-------+---------------+------------+----------------+---------------+---------------+----------------+--------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+-------+-------+------------------+------------------+
|1958|08 Jun 1958 - 14:00 |Group 3|Rasunda Stadium| 

In [49]:
#2
df.filter(df.Stage=='Final').show()
# Returns all the final match games 

+----+--------------------+-----+--------------------+------------------+-----------+---------------+---------------+--------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+---------+------------------+------------------+
|Year|            Datetime|Stage|             Stadium|              City|  Home_Team|Home_Team_Goals|Away_Team_Goals|     Away_Team|      Win_conditions|Attendance|Half_time_Home_Goals|Half_time_Away_Goals|             Referee|         Assistant_1|         Assistant_2| RoundID|  MatchID|Home_Team_Initials|Away_Team_Initials|
+----+--------------------+-----+--------------------+------------------+-----------+---------------+---------------+--------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+---------+------------------+------------------+
|1930|30 Jul 1930 -

In [87]:
#3
USA1 = df.filter( df.Home_Team == 'USA') 
USA2 = df.filter( df.Away_Team == 'USA')  
# returns the number of games that the USA has played in.
print(USA1.count() + USA2.count())

34


In [89]:
#4 
df.filter(df.Stage=='Final').describe('Attendance').show()
# gives the mean min and max and standard deviation for attendence at all final match games 

+-------+------------------+
|summary|        Attendance|
+-------+------------------+
|  count|                20|
|   mean|          76383.65|
| stddev|17836.376878004416|
|    min|            107412|
|    max|             96924|
+-------+------------------+



In [99]:
#5 
df.groupBy('Stadium').sum('Home_Team_Goals').orderBy('sum(Home_Team_Goals)',ascending=False).dropna().show()

# this returns the sum of the home team goals grouped by staduim and then sorted by number of goals 

+--------------------+--------------------+
|             Stadium|sum(Home_Team_Goals)|
+--------------------+--------------------+
|      Estadio Azteca|                39.0|
|  Estadio Centenario|                36.0|
|           St. Jakob|                33.0|
|Maracan� - Est�di...|                26.0|
|             Jalisco|                25.0|
|            Hardturm|                25.0|
|    Parc des Princes|                24.0|
|Nou Camp - Estadi...|                24.0|
|Estadio Ol�mpico ...|                24.0|
|            Nacional|                21.0|
|     Carlos Dittborn|                20.0|
|     Rasunda Stadium|                20.0|
|     Stade V�lodrome|                19.0|
|          Charmilles|                17.0|
|      Parque Central|                16.0|
|Arroyito - Estadi...|                15.0|
|Stade du Parc Les...|                15.0|
|          Nya Ullevi|                15.0|
|       Goodison Park|                15.0|
|   Estadio Sausalito|          